# Telegram Bot Game - Say The Same Thing

https://www.youtube.com/watch?v=2sP1DqyagXE

Bot rules:

We send a new word at the same time.

Every time you and me guess a new word by combing 2 previous words.

If we guessed the same word - we WIN. If not - we continue until one of us give up...

The first pair of words is random.

In [ ]:
! pip install pytelegrambotapi

     |████████████████████████████████| 71kB 2.0MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-3.7.1-cp36-none-any.whl size=50883 sha256=5a629992f4b4a53d5f4d6f7dfcff96718fa9eb9c3cbacb235eca86569e0a01e9
  Stored in directory: /root/.cache/pip/wheels/cd/b2/2c/eac6af3343b21f907123ce013d20ad5ad70c2c3731072d98bf
Successfully built pytelegrambotapi


In [ ]:
import telebot
from gensim.models import KeyedVectors
import numpy as np
from random import choice as random_choice
from sklearn.metrics.pairwise import cosine_similarity

Загрузим обученную модель:

In [ ]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-06-17 07:36:38--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.89.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.89.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  92.8MB/s    in 18s     

2020-06-17 07:36:56 (89.6 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
word2vec = KeyedVectors.load_word2vec_format(fname='GoogleNews-vectors-negative300.bin.gz', limit=200000, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
print(list(word2vec.vocab.keys())[500:700])

['doing', 'face', 'low', 'higher', 'site', 'once', 'yet', 'hours', 'America', 'control', 'received', 'rate', 'career', 'Bush', 'teams', 'known', 'offer', 'race', 'ever', 'experience', 'playing', 'name', 'possible', 'countries', 'Mr.', 'average', 'together', 'using', '9', 'cut', 'While', 'total', 'round', 'young', 'nearly', 'shares', 'member', 'campaign', 'media', 'needs', 'why', 'house', 'issues', 'costs', 'fire', '##-#', 'victory', 'player', 'began', 'sure', 'story', 'per_cent', 'North', 'His', 'staff', 'order', 'war', 'large', 'interest', 'stock', 'food', 'research', 'key', 'India', 'South', 'morning', 'conference', 'senior', 'global', 'Center', 'death', 'person', 'thought', 'gave', 'feel', 'energy', 'history', 'recently', 'largest', 'No.', 'general', 'official', 'released', 'wanted', 'meet', 'short', 'outside', 'running', 'live', 'ball', 'online', 'real', 'position', 'fact', 'fell', 'nine', 'December', 'front', 'action', 'defense', 'problem', 'problems', 'Mr', 'nation', 'needed', 's

Стэммер:

In [ ]:
# from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()

print(stemmer.stem('Miracle'))
print(stemmer.stem('Miraculous'))

print(stemmer.stem('puppy'))
print(stemmer.stem('puppys'))

mirac
mirac
puppy
puppy


Игровой бот:

In [ ]:
API_TOKEN = 'XXXXXXXXXXXXXXXXXXXXXXXX'
history = {}
collection = {}
first_word = list(word2vec.vocab.keys())[500:700]
stemmer = LancasterStemmer() # выделение корня

In [ ]:
bot = telebot.TeleBot(API_TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
  bot.reply_to(message, "This game is called <Say The Same Thing>... Let's start! Send me any english word.")
  if message.chat.id not in history.keys():
    history[message.chat.id] = []
    collection[message.chat.id] = set()


@bot.message_handler(commands=['help'])
def send_welcome(message):
  rules =  'The rules are simple:\n'
  rules += 'We send a new word at the same time.\n'
  rules += 'Then every time you and me guess a new word by combing 2 previous words.\n'
  rules += 'If we guess the same word - we WIN. If not - we continue until one of us give up...\n'
  rules += 'The first pair of words is random.'
  bot.reply_to(message, rules)
  if message.chat.id not in history.keys():
    history[message.chat.id] = []
    collection[message.chat.id] = set()


@bot.message_handler(commands=['replay'])
def replay(message):
  bot.reply_to(message, "OK, let's start again! Send me any english word.")
  history[message.chat.id] = []
  collection[message.chat.id] = set()


@bot.message_handler(commands=['used_stems'])
def used_stems(message):
  used_stems = ', '.join(collection[message.chat.id])
  bot.reply_to(message, "Used stems:\n{}".format(used_stems))


@bot.message_handler(func=lambda message: True)
def echo_message(message):
  chat_id = message.chat.id

  if chat_id not in history.keys(): # если пользователя нет в базе
    bot.reply_to(message, "It seems you're new here or I forgot our last game... Let's start anyway!")
    history[chat_id] = []
    collection[chat_id] = set()

  user_word = message.text.lower()
  if user_word not in word2vec: # если неизвестное слово
    bot.reply_to(message, "I don't know your word <{}>, please try another...".format(user_word))
  elif stemmer.stem(user_word) in collection[message.chat.id]: # если пользователь повторяет слово
    bot.reply_to(message, "We've already used the word <{}> or it's other form, please try another...".format(user_word))
  else:
      if history[chat_id] == []: # если первая игра
        nn_word = random_choice(first_word).lower()
        reply_text = 'you: {:^15}\nme: {:^15}'.format(user_word, nn_word)
      else: # продолжение игры
        old = history[chat_id]
        nn_word = gen_word(chat_id)
        user_sim = get_cos_sim(old[0], old[1], user_word)
        nn_sim = get_cos_sim(old[0], old[1], nn_word)
        reply_text = '{} + {} =\nyou: {:^15} {:.3f}\nme: {:^15}  {:.3f}'.format(
            old[0], old[1], user_word, user_sim, nn_word, nn_sim)
      
      bot.reply_to(message, reply_text)
      # print(reply_text) # подглядывание
      history[chat_id] = [user_word, nn_word]
      collection[chat_id].add(stemmer.stem(user_word))
      collection[chat_id].add(stemmer.stem(nn_word))

      if nn_word == 'I give up': # если сетка не смогла ничего предложить
        bot.reply_to(message, 'I am sorry, I give up... :(')
        replay(message)
      
      if stemmer.stem(user_word) == stemmer.stem(nn_word): # если выиграли
        bot.reply_to(message, 'HOOORAY WE WON in {} steps!'.format(len(collection[chat_id])//2+1))
        replay(message)


def gen_word(chat_id):
  old = history[chat_id]  # old[0] - user_word / old[1] - nn_word
  vec = ( word2vec[old[0]] + word2vec[old[1]] ) / 2
  top10 = word2vec.similar_by_vector(vec, topn=20)

  nn_word = ''
  for word, cos_sim in top10: # выбор неиспользованного слова
    if ('_' not in word) and ('*' not in word) and (cos_sim>0.5):
      if stemmer.stem(word) not in collection[chat_id]:
        nn_word = word.lower()
        break

  if nn_word == '': # проверка существования слова
    nn_word = 'I give up'

  return nn_word


def get_cos_sim(word1, word2, word3):
  if (word1 in word2vec) and (word2 in word2vec) and (word3 in word2vec):
    vec12 = np.array( word2vec[word1] + word2vec[word2] ) / 2
    vec3 = np.array(word2vec[word3])
    return cosine_similarity([vec12],[vec3])[0][0]
  else:
    return 0

Запускаем бота:

In [ ]:
bot.polling()

In [ ]:
history

In [ ]:
collection